could you please please make this into a text with a chapter for the repo gh "#!/usr/bin/env python3
"""
Scans 1000 Genomes chr19 VCF for variants overlapping APOE/TOMM40 3'UTRs and
predicts miRNA seed gain/loss by comparing reference and alternate sequences.

Requirements:
    pip install cyvcf2 biopython pandas intervaltree tqdm
Run:
    python apoe_mirna_scan.py

Outputs:
    results_apoe_miRNA_seed_gain_loss.csv
"""

"

In [1]:
import os
import sys
from cyvcf2 import VCF
from Bio import SeqIO
from Bio.Seq import Seq
import pandas as pd
from intervaltree import IntervalTree
from tqdm import tqdm

In [12]:
# -------- CONFIG --------
DATA_DIR = "data"
VCF_PATH = os.path.join(DATA_DIR, "ALL.chr19.phase3.vcf.gz")
REF_FASTA = os.path.join(DATA_DIR, "chr19.fa")
GTF_PATH = os.path.join(DATA_DIR, "gencode.v19.annotation.gtf")
MATURE_FA = os.path.join(DATA_DIR, "mature.fa")

# genes of interest on chr19 (APOE region)
GENES = ["APOE", "TOMM40"]  # i am focusing on APOE/TOMM40 locus on chr19
# window around variant to check for seed matches (bp)
WINDOW = 30

# -------- helpers --------
def parse_gtf_get_utrs(gtf_path, genes_of_interest):
    """
    Returns dict: gene -> list of (start, end, strand) of 3' UTRs (1-based inclusive).
    GTF is assumed to be GENCODE v19 format (hg19).
    """
    gene_utrs = {g: [] for g in genes_of_interest}
    with open(gtf_path, "rt") as fh:
        for line in fh:
            if line.startswith("#"):
                continue
            parts = line.strip().split("\t")
            if len(parts) < 9:
                continue
            chrom, src, feature, start, end, score, strand, frame, attr = parts
            # if feature != "three_prime_utr":
            #     continue
            # parse gene_name from attributes
            attrs = {}
            for kv in attr.strip().split(";"):
                kv = kv.strip()
                if kv == "":
                    continue
                if " " in kv:
                    k, v = kv.split(" ", 1)
                    attrs[k] = v.strip().strip('"')
            gene_name = attrs.get("gene_name") or attrs.get("gene_id")
            if gene_name in gene_utrs and chrom in ("19", "chr19"):
                s = int(start)
                e = int(end)
                gene_utrs[gene_name].append((s, e, strand))
    return gene_utrs

def build_interval_tree(intervals):
    it = IntervalTree()
    for (s,e,_) in intervals:
        # convert to half-open [s, e) for IntervalTree
        it[s-1:e] = True
    return it

def load_miRNA_seeds(mature_fa_path, species_tag="hsa"):
    """
    Load mature.fa, extract human miRNAs (header contains species), produce seed (nt 2-8).
    Returns dict: miRNA_name -> seed (7mer sequence, 5'->3').
    """
    seeds = {}
    for rec in SeqIO.parse(mature_fa_path, "fasta"):
        # header like: >hsa-let-7a-5p MI0000060 ...
        header = rec.id
        if header.startswith("hsa-") or header.startswith("hsa"):
            name = rec.id
            seq = str(rec.seq).upper()
            if len(seq) >= 8:
                # canonical seed positions 2-8 (1-based)
                seed = seq[1:8]  # python indexing [1:8] => positions 2..8 inclusive
                seeds[name] = seed
    return seeds

def revcomp(seq):
    return str(Seq(seq).reverse_complement())

def sequence_window_from_fasta(chrom_seq, pos1_based, window=30):
    """
    chrom_seq: the chromosome sequence string (1-based indexing assumed)
    pos1_based: variant position (1-based)
    returns sequence of length=window*2+1 centered on pos
    """
    L = len(chrom_seq)
    start = max(1, pos1_based - window)
    end = min(L, pos1_based + window)
    # convert to 0-based for python slicing
    return chrom_seq[start-1:end]

# -------- main --------
def main():
    # ----1) parse GTF --> get 3'UTR intervals for genes of interest
    print("Parsing GTF for 3'UTR coordinates (genes: {})...".format(", ".join(GENES)))
    gene_utrs = parse_gtf_get_utrs(GTF_PATH, GENES)
    for g in GENES:
        print(f"  {g}: {len(gene_utrs.get(g,[]))} 3'UTR segments")

    # build combined interval tree
    combined_intervals = []
    for g, segs in gene_utrs.items():
        for (s,e,strand) in segs:
            combined_intervals.append((s,e,g,strand))
    if not combined_intervals:
        print("No 3'UTR intervals found for genes. Exiting.")
        sys.exit(1)

    tree = IntervalTree()
    for s,e,g,strand in combined_intervals:
        tree[s-1:e] = (g, strand)  # store gene and strand

    # ----2) load chr19 reference sequence
    print("Loading chr19 reference FASTA...")
    chrom_record = None
    for rec in SeqIO.parse(REF_FASTA, "fasta"):
        # fasta header might be ">chr19" or "19"
        if rec.id in ("19","chr19","NC_000019.9"):
            chrom_record = rec
            break
        # try contains 'chr19'
        if "chr19" in rec.id:
            chrom_record = rec
            break
    if chrom_record is None:
        # fallback: take the first record (should be chr19)
        chrom_record = next(SeqIO.parse(REF_FASTA, "fasta"))
    chrom_seq = str(chrom_record.seq).upper()
    print("Reference length:", len(chrom_seq))

    # ----3) load miRNA seeds (human only)
    print("Loading miRNA seeds from miRBase mature.fa ...")
    seeds = load_miRNA_seeds(MATURE_FA)
    print("Loaded {} human miRNA seeds".format(len(seeds)))

    # Build set of seed reverse complements (the sequence in mRNA that matches miRNA seed)
    seed_rc_map = {}  # seed_rc -> list of miRNA names
    for name, seed in seeds.items():
        rc = revcomp(seed)
        seed_rc_map.setdefault(rc, []).append(name)

    # ----4) iterate variants in chr19 VCF and check overlap with 3'UTRs
    print("Scanning VCF for variants overlapping 3'UTRs...")
    vcf = VCF(VCF_PATH)
    results = []
    # small optimization: we iterate through entire chr19 but only evaluate variants whose pos is in tree
    for var in tqdm(vcf):
        pos = var.POS  # 1-based
        hits = tree[pos-1]
        if not hits:
            continue
        # variant may be multi-allelic; process each alt allele separately
        ref = var.REF
        alts = var.ALT
        info = var.INFO
        # get global AF if present (from INFO or compute later)
        af = var.INFO.get('AF') if 'AF' in var.INFO else None

        for alt in alts:
            # build reference window
            seq_ref_window = sequence_window_from_fasta(chrom_seq, pos, window=WINDOW)
            # create alt sequence window by replacing reference base(s) with alt at center
            # determine relative center index in seq_ref_window
            left_len = min(WINDOW, pos-1)
            center_idx = left_len  # 0-based index inside seq_ref_window corresponding to REF start
            # for indels, handle simply by constructing an alt sequence replacement
            # Build local sequence as: left + ref + right
            left = seq_ref_window[:center_idx]
            right = seq_ref_window[center_idx+len(ref):]
            alt_seq_window = left + alt + right

            # search for seed RC matches in both sequences
            gained = []
            lost = []
            # naive string search: check all rc seeds
            for rc_seed, mirnas in seed_rc_map.items():
                if rc_seed in seq_ref_window and rc_seed not in alt_seq_window:
                    # seed lost
                    lost.extend(mirnas)
                if rc_seed not in seq_ref_window and rc_seed in alt_seq_window:
                    # seed gained
                    gained.extend(mirnas)

            # deduplicate and keep counts
            gained_set = sorted(set(gained))
            lost_set = sorted(set(lost))

            results.append({
                "CHROM": var.CHROM,
                "POS": pos,
                "REF": ref,
                "ALT": alt,
                "GENE": ";".join(sorted(set([h.data[0] for h in hits]))),
                "STRAND": ";".join(sorted(set([h.data[1] for h in hits]))),
                "AF": af,
                "GAIN_miRNAs": "|".join(gained_set),
                "LOSS_miRNAs": "|".join(lost_set),
                "n_GAIN": len(gained_set),
                "n_LOSS": len(lost_set),
                "ID": var.ID if var.ID else ".",
            })

    print("Total candidate variant rows:", len(results))
    df = pd.DataFrame(results)
    out_csv = "results_apoe_miRNA_seed_gain_loss.csv"
    df.to_csv(out_csv, index=False)
    print("Wrote", out_csv)
    # summary: top candidates by (n_GAIN + n_LOSS)
    if not df.empty:
        df['impact'] = df['n_GAIN'] + df['n_LOSS']
        top = df.sort_values(by='impact', ascending=False).head(20)
        print("\nTop candidate variants (top 20 by predicted seed gain/loss):")
        print(top[["CHROM","POS","REF","ALT","GENE","n_GAIN","n_LOSS","AF","GAIN_miRNAs","LOSS_miRNAs"]].to_string(index=False))

if __name__ == "__main__":
    main()


Parsing GTF for 3'UTR coordinates (genes: APOE, TOMM40)...
##description: evidence-based annotation of the human genome (GRCh37), version 19 (Ensembl 74)

##provider: GENCODE

##contact: gencode@sanger.ac.uk

##format: gtf

##date: 2013-12-05

  APOE: 48 3'UTR segments
  TOMM40: 134 3'UTR segments
Loading chr19 reference FASTA...
Reference length: 59128983
Loading miRNA seeds from miRBase mature.fa ...
Loaded 2656 human miRNA seeds
Scanning VCF for variants overlapping 3'UTRs...


1832506it [02:17, 13315.60it/s]

Total candidate variant rows: 578
Wrote results_apoe_miRNA_seed_gain_loss.csv

Top candidate variants (top 20 by predicted seed gain/loss):
CHROM      POS REF ALT   GENE  n_GAIN  n_LOSS   AF                                                                                                                                                                                           GAIN_miRNAs                                                                                                                                                                                                               LOSS_miRNAs
   19 45411748   C   T   APOE      12       9 None                           hsa-miR-1207-5p|hsa-miR-1827|hsa-miR-3612|hsa-miR-4270|hsa-miR-4441|hsa-miR-4763-3p|hsa-miR-650|hsa-miR-6754-5p|hsa-miR-6760-5p|hsa-miR-6808-5p|hsa-miR-6893-5p|hsa-miR-940                                                                                  hsa-miR-10396b-5p|hsa-miR-1908-5p|hsa-miR-3175|hsa-miR-4706|hsa